# Inverse Probability Weighting and Doubly Robust Estimation

Prof. Daniel de Abreu Pereira Uhr

### Conteúdo

* Propensity Score Weighting
  * Aplicação no Python
* Doubly Robust Estimation
  * Aplicação no Python




**Observações:** O material apresentado aqui é uma adaptação do material de aula do Prof. Daniel de Abreu Pereira Uhr, e não deve ser utilizado para fins comerciais. O material é disponibilizado para fins educacionais e de pesquisa, e não deve ser reproduzido sem a devida autorização do autor. Este material pode conter erros e imprecisões. O autor não se responsabiliza por quaisquer danos ou prejuízos decorrentes do uso deste material. O uso deste material é de responsabilidade exclusiva do usuário. Caso você encontre erros ou imprecisões neste material, por favor, entre em contato com o autor para que possam ser corrigidos. O autor agradece qualquer *feedback* ou sugestão de melhoria.

---

### Propensity Score Weighting

***Definição formal do Propensity Score***

O **Propensity Score** é definido como a probabilidade condicional de uma unidade receber o tratamento dado o vetor de covariáveis observadas:

$$
p(X_i) = \Pr(D_i = 1 \mid X_i)
$$

onde:

- $D_i \in \{0, 1\}$ é o indicador de tratamento;
- $X_i$ é o vetor de covariáveis observadas.

A ideia central de **Propensity Score Weighting** é reponderar a amostra de forma a criar uma população *pseudo‐randomizada*, em que a distribuição das covariáveis seja balanceada entre tratados e não tratados.

### Inverse Probability of Treatment Weight (IPTW)

***Pesos para o ATE (Average Treatment Effect)***

O peso **IPTW** para o ATE é definido como:

$$
w_i^{ATE} = \frac{D_i}{p(X_i)} + \frac{1 - D_i}{1 - p(X_i)}
$$

**Na prática**:
- Tratados ($D_i = 1$) recebem peso $1 / p(X_i)$  
- Controles ($D_i = 0$) recebem peso $1 / (1 - p(X_i))$


***Pesos para o ATT (Average Treatment Effect on the Treated)***

O peso **IPTW** para o ATT é definido como:

$$
w_i^{ATT} = D_i + (1 - D_i) \cdot \frac{p(X_i)}{1 - p(X_i)}
$$

**Na prática**:
- Tratados ($D_i = 1$) recebem peso = 1  
- Controles ($D_i = 0$) recebem peso $p(X_i) / (1 - p(X_i))$

***Pesos para o ATU (Average Treatment Effect on the Untreated)***

O peso **IPTW** para o ATU é definido como:

$$
w_i^{ATU} = (1 - D_i) + D_i \cdot \frac{1 - p(X_i)}{p(X_i)}
$$

**Na prática**:
- Tratados ($D_i = 1$) recebem peso $1 - p(X_i) / p(X_i)$
- Controles ($D_i = 0$) recebem peso = 1


***Interpretação dos pesos***

- **No ATE**:  
  - Unidades tratadas com baixa probabilidade de tratamento recebem peso alto.  
  - Unidades de controle com baixa probabilidade de não tratamento recebem peso alto.  

- **No ATT**:  
  - Tratados sempre têm peso 1.  
  - Controles são reponderados para se assemelhar à distribuição de covariáveis dos tratados.

- **No ATU**:  
  - Tratados são reponderados para se assemelhar à distribuição de covariáveis dos não tratados.
  - Controles sempre têm peso 1.


### Hipóteses de identificação

1. **Ignorabilidade (ou não confusão)**:  
   $$
   (Y_i(1), Y_i(0)) \perp D_i \mid X_i
   $$  
   Isto é, não existem confundidores não observados dados $X_i$.

2. **Sobreposição (ou suporte comum)**:  
   $$
   0 < p(X_i) < 1 \quad \forall i
   $$  
   Cada unidade tem probabilidade positiva de receber ambos os estados de tratamento.

### Estimação do efeito causal ponderado

O **ATE ponderado** pode ser visto como:

> Média ponderada dos resultados observados entre os tratados  
> **menos**  
> Média ponderada dos resultados observados entre os controles.

Matematicamente (Horvitz–Thompson - não normalizadas)):




Ou Normalizando (Hajek / self-normalized):

$$
\widehat{\tau}_{ATE} =
\underbrace{\frac{\sum_{i=1}^n w_i^{ATE} D_i Y_i}{\sum_{i=1}^n w_i^{ATE} D_i}}_{\text{Média ponderada nos tratados}}
-
\underbrace{\frac{\sum_{i=1}^n w_i^{ATE} (1-D_i) Y_i}{\sum_{i=1}^n w_i^{ATE} (1-D_i)}}_{\text{Média ponderada nos controles}}
$$

* O **ATT** pode ser obtido substituindo $w_i^{ATE}$ por $w_i^{ATT}$ na fórmula acima, interpretando o resultado como efeito médio nos tratados.
* O **ATU** pode ser obtido substituindo $w_i^{ATE}$ por $w_i^{ATU}$ na fórmula acima, interpretando o resultado como efeito médio nos não tratados.


### Aplicação em Python

In [1]:
import pandas as pd
import numpy as np
import statsmodels.formula.api as smf

In [2]:
# DataFrame
df = pd.read_stata("https://github.com/Daniel-Uhr/data/raw/main/cattaneo2.dta")

In [3]:
# Criar a variável de resultado
df['Y'] = df['bweight']

# Crie a variável 'Treated' com valor inicial de 0
df['Treated'] = 0
# Recodifique 'Treated' para 1 se 'mbsmoke' for igual a 'smoker'
df.loc[df['mbsmoke'] == 'smoker', 'Treated'] = 1

df['casada'] = 0
df.loc[df['mmarried']=='married', 'casada'] = 1

# gerar uma variável de contagem de linhas iniciando em 1
df['id'] = np.arange(len(df)) + 1

# Conjunto de Covariáveis X
X = ['casada', 'mage', 'medu']

Rodar o escore de propensão e salvar no dataframe "pscore":

In [4]:
# Estimar o escore de propensão com regressão logística
df['pscore'] = smf.logit("Treated ~ casada + mage + medu", data=df).fit().predict()

Optimization terminated successfully.
         Current function value: 0.446546
         Iterations 6


In [31]:
df[['Y', 'Treated', 'pscore']]

,Y,Treated,pscore
0,3459,0,0.106292
1,3260,0,0.358668
2,3572,0,0.190573
3,2948,0,0.139211
4,2410,0,0.130666
...,...,...,...
4637,3317,1,0.299040
4638,3030,1,0.145164
4639,2950,0,0.304181
4640,3969,0,0.134881


Criar os pesos:

In [ ]:
# ATE weights
df['W_ATE_T'] = df['Treated'] / df['pscore']
df['W_ATE_C'] = (1 - df['Treated']) / (1 - df['pscore'])

In [35]:
df[['Y', 'Treated', 'pscore', 'W_ATE_T', 'W_ATE_C']]

,Y,Treated,pscore,W_ATE_T,W_ATE_C
0,3459,0,0.106292,0.000000,1.118933
1,3260,0,0.358668,0.000000,1.559254
2,3572,0,0.190573,0.000000,1.235442
3,2948,0,0.139211,0.000000,1.161724
4,2410,0,0.130666,0.000000,1.150306
...,...,...,...,...,...
4637,3317,1,0.299040,3.344033,0.000000
4638,3030,1,0.145164,6.888768,0.000000
4639,2950,0,0.304181,0.000000,1.437156
4640,3969,0,0.134881,0.000000,1.155911


In [42]:
df['Y_psw_ATE'] = df['Y'] * df['W_ATE_T'] - df['Y'] * df['W_ATE_C']

In [44]:
df[['Y', 'Treated', 'pscore', 'W_ATE_T', 'W_ATE_C', 'Y_psw_ATE']]

,Y,Treated,pscore,W_ATE_T,W_ATE_C,Y_psw_ATE
0,3459,0,0.106292,0.000000,1.118933,-3870.390175
1,3260,0,0.358668,0.000000,1.559254,-5083.167326
2,3572,0,0.190573,0.000000,1.235442,-4413.000460
3,2948,0,0.139211,0.000000,1.161724,-3424.763767
4,2410,0,0.130666,0.000000,1.150306,-2772.237288
...,...,...,...,...,...,...
4637,3317,1,0.299040,3.344033,0.000000,11092.156261
4638,3030,1,0.145164,6.888768,0.000000,20872.967997
4639,2950,0,0.304181,0.000000,1.437156,-4239.609565
4640,3969,0,0.134881,0.000000,1.155911,-4587.809591


In [45]:
df['Y_psw_ATE'].mean()

-361.2246611411922

***ATT psw***

In [47]:
# ATT weights
df['W_ATT_T'] = df['Treated'] * 1 
df['W_ATT_C'] = (1 - df['Treated']) * (df['pscore'] / (1 - df['pscore']))

In [48]:
df[['Y', 'Treated', 'pscore', 'W_ATT_T', 'W_ATT_C']]

,Y,Treated,pscore,W_ATT_T,W_ATT_C
0,3459,0,0.106292,0,0.118933
1,3260,0,0.358668,0,0.559254
2,3572,0,0.190573,0,0.235442
3,2948,0,0.139211,0,0.161724
4,2410,0,0.130666,0,0.150306
...,...,...,...,...,...
4637,3317,1,0.299040,1,0.000000
4638,3030,1,0.145164,1,0.000000
4639,2950,0,0.304181,0,0.437156
4640,3969,0,0.134881,0,0.155911


In [49]:
df['Y_psw_ATT'] = df['Y'] * df['W_ATT_T'] - df['Y'] * df['W_ATT_C']

In [50]:
df['Y_psw_ATT'].mean()

-54.25077509603716

***ATU psw***

In [55]:
# ATU weights
df['W_ATU_T'] = df['Treated'] * (1 - df['pscore']) / df['pscore']
df['W_ATU_C'] = (1 - df['Treated']) * 1

In [56]:
df[['Y', 'Treated', 'pscore', 'W_ATU_T', 'W_ATU_C']]

,Y,Treated,pscore,W_ATU_T,W_ATU_C
0,3459,0,0.106292,0.000000,1
1,3260,0,0.358668,0.000000,1
2,3572,0,0.190573,0.000000,1
3,2948,0,0.139211,0.000000,1
4,2410,0,0.130666,0.000000,1
...,...,...,...,...,...
4637,3317,1,0.299040,2.344033,0
4638,3030,1,0.145164,5.888768,0
4639,2950,0,0.304181,0.000000,1
4640,3969,0,0.134881,0.000000,1


In [57]:
df['Y_psw_ATU'] = df['Y'] * df['W_ATU_T'] - df['Y'] * df['W_ATU_C']

In [58]:
df['Y_psw_ATU'].mean()

-306.9738860451551

**IPW Normalizado**

É comum utilizar a normalização dos pesos de IPW para garantir que a soma dos pesos seja igual a 1. Isso é feito dividindo cada peso pela soma total dos pesos. Isso também ajuda a reduzir a variância dos estimadores ponderados. É conhecido como a forma de Hájek para o PSW.

In [59]:
df['Y_psw_ATE_normalized'] = (df['Y'] * df['W_ATE_T'] / df['W_ATE_T'].sum()) - (df['Y'] * df['W_ATE_C'] / df['W_ATE_C'].sum())

In [62]:
df[['Y', 'Treated', 'pscore', 'Y_psw_ATE_normalized']]

,Y,Treated,pscore,Y_psw_ATE_normalized
0,3459,0,0.106292,-0.830172
1,3260,0,0.358668,-1.090305
2,3572,0,0.190573,-0.946558
3,2948,0,0.139211,-0.734588
4,2410,0,0.130666,-0.594626
...,...,...,...,...
4637,3317,1,0.299040,2.477149
4638,3030,1,0.145164,4.661442
4639,2950,0,0.304181,-0.909367
4640,3969,0,0.134881,-0.984054


Para recuperar o efeito quando utilizamos os pesos normalizados somamos as contribuições individuais.

In [61]:
df['Y_psw_ATE_normalized'].sum()

-234.43712533815778

As duas são estimadores diferentes do mesmo alvo (ATE, ATT, ATU).

* HT é não-normalizado, pode ter mais variância, mas é não-viesado sob overlap.
* Hájek é normalizado, tende a ser mais estável em amostras finitas, mas pode ter um viés pequeno.

Por isso o número pode mudar bastante — principalmente se você tiver pesos extremos (como você já viu: alguns > 10). O Hájek “reescala” as contribuições, o que pode puxar o efeito para outro valor.

### Aplicando no Python

Para realizar a aplicação direta da ponderação por IPW, com efeito médio do tratamento e seus intervalos de confiança, podemos utilizar a biblioteca `statsmodels` para ajustar um modelo de regressão ponderada.


Aqui calculamos os pesos para os indivíduos tratados e não tratados em uma variável no dataframe.

In [ ]:
# ATE weights no python
df['W_ATE_ols'] = df['Treated'] / df['pscore'] + (1 - df['Treated']) / (1 - df['pscore'])

Depois rodamos a regressão

In [66]:
# Aplicando regressão ponderada (IPTW) para ATE
model_ate = smf.wls("Y ~ Treated", data=df, weights=df['W_ATE_ols']).fit()
print(model_ate.summary())

                            WLS Regression Results                            
Dep. Variable:                      Y   R-squared:                       0.040
Model:                            WLS   Adj. R-squared:                  0.040
Method:                 Least Squares   F-statistic:                     194.9
Date:                qui, 21 ago 2025   Prob (F-statistic):           2.03e-43
Time:                        11:32:25   Log-Likelihood:                -36594.
No. Observations:                4642   AIC:                         7.319e+04
Df Residuals:                    4640   BIC:                         7.321e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept   3401.1645     11.754    289.358      0.0

Repare que uma propriedade importante do WLS (Weighted Least Squares), é que ele já faz a normalização implícita (ele força as médias ponderadas a se alinharem com os pesos), mesmo que nós tenhamos indicado pesos diferentes.

Para o ATT:

In [67]:
# ATT weights
df['W_ATT_ols'] = df['Treated'] * 1  + (1 - df['Treated']) * (df['pscore'] / (1 - df['pscore']))

ATE com peso IPTW apresentou efeito de -234.4371 gramas.

In [68]:
# Aplicando regressão ponderada (IPTW) para ATT
model_att = smf.wls("Y ~ Treated", data=df, weights=df['W_ATT_ols']).fit()
print(model_att.summary())

                            WLS Regression Results                            
Dep. Variable:                      Y   R-squared:                       0.032
Model:                            WLS   Adj. R-squared:                  0.032
Method:                 Least Squares   F-statistic:                     155.8
Date:                qui, 21 ago 2025   Prob (F-statistic):           3.41e-35
Time:                        11:35:03   Log-Likelihood:                -37060.
No. Observations:                4642   AIC:                         7.412e+04
Df Residuals:                    4640   BIC:                         7.414e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept   3350.9689     12.014    278.924      0.0

ATT com peso IPTW normalizado é -213.3092

***ATU OLS***

In [69]:
df['W_ATU_ols'] = df['Treated'] * (1 - df['pscore']) / df['pscore'] + (1 - df['Treated']) * 1

In [70]:
# Aplicando regressão ponderada (IPTW) para ATU
model_atu = smf.wls("Y ~ Treated", data=df, weights=df['W_ATU_ols']).fit()
print(model_atu.summary())

                            WLS Regression Results                            
Dep. Variable:                      Y   R-squared:                       0.042
Model:                            WLS   Adj. R-squared:                  0.042
Method:                 Least Squares   F-statistic:                     205.0
Date:                qui, 21 ago 2025   Prob (F-statistic):           1.58e-45
Time:                        11:36:38   Log-Likelihood:                -36580.
No. Observations:                4642   AIC:                         7.316e+04
Df Residuals:                    4640   BIC:                         7.318e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept   3412.9116     11.684    292.111      0.0

### Doubly Robust Estimation (DR - AIPW)

***Doubly Robust Estimation (DR)***

* É a ideia geral de combinar Outcome Regression e Inverse Probability Weighting. Um estimador é “duplamente robusto” quando é consistente se pelo menos um dos modelos (de resultado ou de propensão) estiver corretamente especificado. Exemplo clássico: AIPW.

***AIPW (Augmented Inverse Probability Weighting)***

* É um caso específico de estimador duplamente robusto. A forma canônica do AIPW para o ATE é exatamente aquela fórmula que vou apresentar.


***Motivação***

Na avaliação de efeitos causais, dois métodos são comuns:  

- **Outcome Regression (OR):** modelar diretamente o resultado condicional a $X$ e $D$.  
- **Inverse Probability Weighting (IPW):** reponderar as observações pelo escore de propensão $\hat{p}(X)$.  

Cada um é **consistente apenas se o modelo estiver corretamente especificado**.  
O **estimador duplamente robusto (Doubly Robust, ou Augmented IPW - AIPW)** combina as duas abordagens, garantindo consistência se **pelo menos um dos modelos estiver correto**.

**Modelos básicos**

- Tratamento: $D_i \in \{0,1\}$
- Covariáveis: $X_i$
- Resultado: $Y_i = D_i Y_i(1) + (1-D_i) Y_i(0)$
- Propensity score: $p(X) = P(D=1|X)$
- Funções de regressão: $\mu_d(X) = E[Y|D=d,X]$

Queremos o efeito médio do tratamento (ATE):

$$
\beta_{ATE} = E[Y(1) - Y(0)] 
$$


#### Outcome Regression (OR)

O estimador para o ATE é:

$$
\hat{\beta}_{OR} = \frac{1}{n}\sum_{i=1}^n \left[ \hat{\mu}_1(X_i) - \hat{\mu}_0(X_i) \right]
$$

onde $\hat{\mu}_d(X)$ é obtido por regressão.


#### Inverse Probability Weighting (IPW)

O estimador ATE via IPW é:

$$
\hat{\beta}_{IPW} = \frac{1}{n}\sum_{i=1}^n \left( \frac{D_i Y_i}{\hat{p}(X_i)} - \frac{(1-D_i)Y_i}{1-\hat{p}(X_i)} \right)
$$


#### Estimador Doubly Robust (AIPW)

O estimador duplamente robusto para o ATE é dado por:

$$
\hat{\beta}_{DR} = \frac{1}{n}\sum_{i=1}^n \Bigg\{ \Big(\hat{\mu}_1(X_i) - \hat{\mu}_0(X_i)\Big) \;+\; \frac{D_i}{\hat{p}(X_i)}\big(Y_i - \hat{\mu}_1(X_i)\big) \;-\; \frac{1-D_i}{1-\hat{p}(X_i)}\big(Y_i - \hat{\mu}_0(X_i)\big) \Bigg\}
$$

**Interpretação:**  

- O **primeiro termo** ($\hat{\mu}_1 - \hat{\mu}_0$) é a predição média da regressão de resultados.  
- Os **termos adicionais** são “correções” baseadas em resíduos ponderados por IPW.  


***Propriedade de Dupla Robustez***

- Se $\hat{\mu}_d(X)$ estiver corretamente especificado → os resíduos têm média zero, e o estimador DR converge para o ATE mesmo que $\hat{p}(X)$ esteja errado.  
- Se $\hat{p}(X)$ estiver corretamente especificado → a ponderação corrige vieses da regressão mal especificada, garantindo consistência.  
- Se **ambos** estiverem corretos → o estimador é **eficiente** (atinge menor variância assintótica).


***Em termos intuitivos:***

- O DR é chamado também de **Augmented IPW** porque começa com IPW e **adiciona regressão dos resultados** como “ajuste extra”.  
- Outra visão: é uma **regressão de resultados com resíduos reponderados**, de modo que o viés de especificação de um modelo é compensado pelo outro.  


**Extensões**

- **ATT (Average Treatment Effect on the Treated):** versão análoga existe, com pesos ajustados para focar nos tratados.  



### Aplicação em Python

Para facilitar a aplicação vamos utilizar uma abordagem moderna chamada ***DR-Learner***.

***DR-Learner***
* É um meta-algoritmo de Machine Learning (Chernozhukov et al., 2018; Nie & Wager, 2021), que usa a ideia de dupla robustez junto com algoritmos de ML para estimar Conditional Average Treatment Effects (CATE). O DR-Learner aproveita a estrutura AIPW (score de influência duplamente robusto) e depois ajusta modelos de ML em cima disso para obter heterogeneidade do efeito. Portanto, DR-Learner é um desdobramento moderno do AIPW aplicado à estimação de efeitos condicionais (CATE), não sinônimo.



## Doubly Robust Estimation (DR)

A estimativa duplamente robusta combina uma forma de "Outcome Regression" com um modelo de ponderação (ou seja, utilizando o escore de propensão) para estimar o efeito causal sobre um resultado. Quando usados individualmente para estimar um efeito causal, os métodos de regressão de resultados e escore de propensão são não enviesados apenas se o modelo estatístico for especificado corretamente. O **estimador duplamente robusto** combina estas 2 abordagens de modo que apenas 1 dos 2 modelos precisa ser especificado corretamente para obter um estimador de efeito não-viesado.

A especificação correta do modelo de regressão é um pressuposto fundamental na análise econométrica. Quando o objetivo é ajustar o fator de confusão, o estimador é consistente (e, portanto, assintoticamente não-enviesado) se o modelo refletir as verdadeiras relações entre a exposição e os fatores de confusão com o resultado. Na prática, nunca poderemos saber se algum modelo específico representa com precisão essas relações. **A estimativa duplamente robusta combina regressão de resultados com ponderação pelo escore de propensão (PS), de modo que o estimador seja robusto à especificação incorreta de um (mas não de ambos) desses modelos**.


**Outcome Regression Approach**

Vimos que:

$$ \hat{\beta}_{ATE}^{OR} = E[\mu_{1}(X) - \mu_{0}(X)] + E[(Y_{1} - \mu_{1}(X)) - (Y_{0} - \mu_{0}(X))] $$

e

$$ \hat{\beta}^{OR}_{ATT} = E[\mu_{1}(X^{1}) - \mu_{0}(X^{1})] + E[(Y_{1} - \mu_{1}(X^{1})) - (Y_{0} - \mu_{0}(X^{1}))] $$




**Approach de Inverse Probability Weighting (IPW)**

Nesta abordagem, o viés de confusão é ajustado por meio de técnicas de matching (pareamento) e ponderação pelo escore de propensão (Peso = $W$). As ponderações são calculadas da seguinte forma:

$$ W_{ATE} = \frac{D}{\hat{p}(X)} + \frac{1-D}{1-\hat{p}(X)} $$

e,

$$ W_{ATT} = D + (1-D)\frac{\hat{p}(X)}{1-\hat{p}(X)} $$


**Approach do Doubly Robust Estimation (DR)**

A abordagem Doubly Robust Estimation (DR) combina as vantagens das abordagens de Outcome Regression e de Ponderação pela Probabilidade Inversa. Isso proporciona uma maior robustez aos resultados. Os estimadores duplamente robustos para o Average Treatment Effect (ATE) e o Average Treatment Effect on the Treated (ATT) são dados pelas seguintes fórmulas:

* **Doubly Robust Estimation for Average Treatment Effect (ATE)**

$$ \hat{\beta_{ATE}^{DR}} =  \mathbb{E} \left[ (\mu_1(X) - \mu_0 (X)) \right] + \mathbb{E} \left[ \frac{D}{\hat{p}(X)}.(Y_{1} - \mu_1 (X)) - \frac{(1-D)}{1-\hat{p}(X)}.(Y_{0} - \mu_0 (X)) \right] $$

* **Doubly Robust Estimation for Average Treatment Effect on the Treated (ATT)**

$$ \hat{\beta_{ATT}^{DR}} = \mathbb{E} \left[ (\mu_1 (X) - \mu_0 (X)) \right] + \mathbb{E} \left[ D(Y_{1} - \mu_1 (X)) - \frac{(1-D)\hat{p}(X)}{1-\hat{p}(X)}.(Y_{0} - \mu_0 (X)) \right] $$


Repare que realizamos um "ajuste" nos resíduos da regressão de resultados, ponderando-os pelo escore de propensão. Isso garante que o estimador seja robusto à especificação incorreta de um dos modelos. O "ajuste" é essencialmente um estimador IPW realizado sobre os resíduos.

Por isso que o Doubly Robust Estimation também é conhecido como Augmented Inverse Probability Weighting (AIPW).

**Por que a o estimador Duplamente Robusto (*Augmented Inverse Probability Weighting* - AIPW) é tão atraente?**

A razão é que só precisamos de uma das duas previsões, *$\hat{\mu}$* ou *$\hat{p}$*, para que a estimativa seja correta (não enviesada/imparcial). 
* Se ambos os modelos estiverem corretos, o estimador será mais eficiente do que qualquer um dos modelos sozinho. 
* Se um dos modelos estiver errado, o estimador ainda será consistente, desde que o outro modelo esteja correto. 

Isso é uma grande vantagem em relação a outras abordagens, como a regressão de resultados ou a ponderação pelo escore de propensão, que exigem que ambos os modelos estejam corretos para que o estimador seja consistente.

Suponha que $\hat{\mu}$ esteja especificado corretamente. Então $E[\hat{\mu}^{d}(x)=E[Y|X=x, D=d]$ , então o estimador DR é consistente, mesmo que o modelo de propensão $\hat{p}$ esteja mal especificado.

$$ \hat{\beta^{DR}} =  \mathbb{E} \left[ (\mu_1(X) - \mu_0 (X)) +  \frac{D}{\hat{p}(X)}.(Y_{1} - \mu_1 (X)) - \frac{(1-D)}{1-\hat{p}(X)}.(Y_{0} - \mu_0 (X)) \right] = $$

$$  =  \mathbb{E} \left[ (\mu_1(X) - \mu_0 (X)) \right] = $$

$$  =  \mathbb{E} \left[ Y^{1} - Y^{0} \right] = $$

$$ = \beta $$

A intuição é que, se $\hat{\mu}$ está **especificado corretamente é imparcial e o fator de ajuste desaparece**, uma vez que os resíduos convergem para zero.


Por outro lado, suponha $\hat{p}$ está especificado corretamente, ou seja, $E[\hat{p}(X)]=P(D=1|X)$, então o estimador DR é consistente, mesmo que o modelo de resultados $\hat{\mu}$ esteja mal especificado.

$$ \hat{\beta^{DR}} =  \mathbb{E} \left[ (\mu_1(X) - \mu_0 (X)) +  \frac{D}{\hat{p}(X)}.(Y_{1} - \mu_1 (X)) - \frac{(1-D)}{1-\hat{p}(X)}.(Y_{0} - \mu_0 (X)) \right] = $$

$$ \mathbb{E} \left[ \mu_1(X) - \mu_0 (X) + \frac{D}{\hat{p}(X)}.Y_{1} - \frac{D}{\hat{p}(X)}.\mu_1 (X) - \frac{(1-D)}{1-\hat{p}(X)}Y_{0} + \frac{(1-D)}{1-\hat{p}(X)}.\mu_0 (X) \right] = $$


$$ \mathbb{E} \left[ \frac{D}{\hat{p}(X)}.Y_{1} - \frac{(1-D)}{1-\hat{p}(X)}Y_{0} + \mu_1(X) - \frac{D}{\hat{p}(X)}.\mu_1 (X) + \frac{(1-D)}{1-\hat{p}(X)}.\mu_0 (X) - \mu_0 (X) \right] = $$


$$ \mathbb{E} \left[ \frac{D}{\hat{p}(X)}.Y_{1} - \frac{(1-D)}{1-\hat{p}(X)}Y_{0} + \mu_1(X) (1 - \frac{D}{\hat{p}(X)}) + \mu_0 (X)(\frac{(1-D)}{1-\hat{p}(X)} - 1) \right] = $$

$$ \mathbb{E} \left[ \frac{D}{\hat{p}(X)}.Y_{1} - \frac{(1-D)}{1-\hat{p}(X)}Y_{0} + \mu_1(X) (\frac{\hat{p}(X) - D}{\hat{p}(X)}) + \mu_0 (X)(\frac{(1-D)- (1-\hat{p}(X))}{1-\hat{p}(X)}) \right] = $$

$$ \mathbb{E} \left[ \frac{D}{\hat{p}(X)}.Y_{1} - \frac{(1-D)}{1-\hat{p}(X)}Y_{0} \right] = $$

$$ \mathbb{E} \left[ Y^{1} - Y^{0} \right] = $$

$$ = \beta $$

A intuição é que, se $\hat{p}$ está especificado corretamente, o $\hat{\beta}^{DR}$ é imparcial e o fator de ajuste desaparece, uma vez que os resíduos ($D_{i}-\hat{p}(X)$) convergem para zero.


### Aplicação em Python

In [ ]:
import statsmodels.formula.api as smf
import seaborn as sns
from sklearn.linear_model import LinearRegression
import numpy as np

In [ ]:
# Regressão Logística para estimar o escore de propensão
logit_model = smf.logit("Treated ~ 1 + casada + mage + medu", data=df).fit()
# Imprimindo o modelo
print(logit_model.summary())

In [ ]:
# Salvando o escore de propensão no DataFrame
df['ps'] = logit_model.predict()

In [ ]:
# Verificar graficamente a área de sobreposição
sns.histplot(data=df, x='ps', hue='Treated', bins=100, stat='density', common_norm=False).\
    set(ylabel="", title="Distribution of Propensity Scores");

Agora vamos calcular os pesos IPW para o ATE e ATT.

In [ ]:
# Inverse Probability of Treatment Weight (IPTW)

# Peso para o efeito médio do tratamento (ATE)
df['W1'] = 1 / df['ps']
df.loc[df['Treated'] == 0, 'W1'] = 0
df['W2'] = 1 / (1 - df['ps'])
df.loc[df['Treated'] == 1, 'W2'] = 0

# Peso para o efeito médio do tratamento nos tratados (ATT)
df['W_ATE'] = df['W1'] + df['W2']
df['W_ATT'] = df['ps'] / (1 - df['ps'])
df.loc[df['Treated'] == 1, 'W_ATT'] = 1

A título de curiosidade, podemos estimar o efeito médio do tratamento para os tratados (ATT) e o efeito médio do tratamento (ATE) utilizando o método de Ponderação pelo Escore de Propensão (IPW). Para isso, basta rodar a regressão linear considerando como peso amostral os valores de $W_{ATE}$ e $W_{ATT}$, respectivamente.

In [ ]:
#Propensity Score Weighting - ATE
psw_ate = smf.wls("Y ~ Treated", weights=df['W_ATE'], data=df).fit()
print(psw_ate.summary())

In [ ]:
#Propensity Score Weighting - ATT
psw_att = smf.wls("Y ~ Treated", weights=df['W_ATT'], data=df).fit()
print(psw_att.summary())

Repare que podemos obter os mesmos resultados utilizando o pacote pyDRReg.

In [ ]:
from pyDRReg.pyDRReg import pyDRReg

T_var = 'Treated'
Y_var = 'Y'
X_vars = ['casada', 'mage', 'medu']

IPW_att = pyDRReg(df, X_vars, T_var, Y_var, method='att', estimator='IPW', n_bootstrap=50, seed=44)

print(IPW_att.summary())

In [ ]:
from pyDRReg.pyDRReg import pyDRReg

T_var = 'Treated'
Y_var = 'Y'
X_vars = ['casada', 'mage', 'medu']

IPW_ate = pyDRReg(df, X_vars, T_var, Y_var, method='ate', estimator='IPW', n_bootstrap=50, seed=44)

print(IPW_ate.summary())

**Estimativa Duplamente Robusta**

Agora temos todos os componentes para estimar o DR para ATE e o ATT, vamos fazer "na mão".

In [ ]:
# DR-ATE
DR_ATE = mu1 - mu0 + df["Treated"] / df["ps"] * (df["Y"] - mu1) - (1-df["Treated"]) / (1-df["ps"]) * (df["Y"] - mu0)
print(np.mean(DR_ATE))

In [ ]:
# DR-ATT
DR_ATT = mu1 - mu0 + df["Treated"] * (df["Y"] - mu1) - (1-df["Treated"])*df["ps"] / (1-df["ps"]) * (df["Y"] - mu0)
print(np.mean(DR_ATT))

Podemos utilizar o pacote pyDRReg para obter os mesmos resultados.

In [ ]:
from pyDRReg.pyDRReg import pyDRReg

T_var = 'Treated'
Y_var = 'Y'
X_vars = ['casada', 'mage', 'medu']

DR_ate = pyDRReg(df, X_vars, T_var, Y_var, method='ate', estimator='DR', n_bootstrap=50, seed=44)

print(DR_ate.summary())

In [ ]:
from pyDRReg.pyDRReg import pyDRReg

T_var = 'Treated'
Y_var = 'Y'
X_vars = ['casada', 'mage', 'medu']

DR_att = pyDRReg(df, X_vars, T_var, Y_var, method='att', estimator='DR', n_bootstrap=50, seed=44)

print(DR_att.summary())

Esse tipo de estimador é bastante importante na literatura. E já possui alguns estimadores que realizam as estimações de forma direta. Por exemplo, poderíamos computar diretamente com 'LinearDRLearner' da biblioteca 'EconML' da Microsoft (EconML - Estimate causal effects with ML).

obs: https://www.microsoft.com/en-us/research/project/econml/

In [ ]:
from sklearn.linear_model import LogisticRegression, LinearRegression
from econml.dr import LinearDRLearner

In [ ]:
X = df[['casada', 'mage', 'medu']]

In [ ]:
model = LinearDRLearner(model_propensity=LogisticRegression(), 
                        model_regression=LinearRegression(),
                        random_state=1)
model.fit(Y=df["Y"], T=df["Treated"], X=X);

In [ ]:
model.ate_inference(X=X.values, T0=0, T1=1).summary().tables[0]

O modelo nos dá diretamente o efeito médio do tratamento. A estimativa é estatisticamente diferente de zero e o intervalo de confiança inclui o valor verdadeiro de -229,17. Observe que obtivemos uma estimativa diferente porque a função **LinearDRLearner** também realizou o cross-fitting em segundo plano, o que não fizemos antes. Ele não calcula o ATT.

Outro pacote importante é o "causalml" (https://causalml.readthedocs.io/en/latest/about.html).



## Boas práticas

* Verifique o balanço das covariáveis.
  * Tanto o IPW quanto o DR (AIPW) foram desenvolvidos para ambientes nos quais o tratamento não é atribuído aleatoriamente incondicionalmente, mas pode depender de algumas variáveis observáveis. Essas informações podem ser verificadas de duas maneiras: 
    * (1) Produza uma tabela de médias/equilíbrio das covariáveis. Se a randomização incondicional não for válida, esperamos ver diferenças significativas entre alguns observáveis; 
    * (2) Trace os escores de propensão estimados. Se a randomização incondicional for válida, esperamos que os escores de propensão sejam constantes.
* Verifique a suposição de sobreposição.
  * Podemos simplesmente verificar os limites dos escores de propensão previstos. Se a suposição de sobreposição for violada, acabamos dividindo algum termo do estimador por zero.